# `Close` Contract on Mainnet Using Marlowe Runtime

This is the first contract executed on `mainnet` solely using the Marlowe Runtime backend.

## Preliminaries

Record version numbers

In [1]:
marlowe-cli --version

marlowe-cli 0.0.10.0 @ 82e72feafb1fcd869c9744fde537527227fafed2 «mainnet»


In [2]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [3]:
git rev-parse HEAD

ef671d84e5bab86c12c6770cf541ada8e8c88b43


### Setup the faucet.

Set the location of keys.

In [4]:
TREASURY=treasury

Set the faucet.

In [5]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.mainnet.address)
echo "$FAUCET_ADDR"

addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h


### Select network

In [6]:
export CARDANO_NODE_SOCKET_PATH=node.socket
export CARDANO_TESTNET_MAGIC=64824073
MAGIC=(--mainnet)
echo "${MAGIC[@]}"

--mainnet


### The party to the contract

In [7]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [8]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx


Fund the address

In [9]:
marlowe-cli util fund-address \
  "${MAGIC[@]}" \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 10000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "bb945a4137b2070ffea5abde1d1ae042e5e51fc6c3c76f5a766ec4a34d02cfca"


### Time computations

In [10]:
SECOND=1000
MINUTE=$((60 * SECOND))

In [11]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1670768142000


## The Contract

We set the parameters for the `Close` contract.

In [12]:
MINIMUM_ADA=2000000

Create the contract.

In [13]:
cat << EOI > runtime-close-1.contract
"close"
EOI
json2yaml runtime-close-1.contract

close
...


Create the initial state.

In [14]:
yaml2json << EOI > runtime-close-1.state
accounts:
- - - address: $PARTY_ADDR
    - currency_symbol: ''
      token_name: ''
  - $MINIMUM_ADA
boundValues: []
choices: []
minTime: 0
EOI
cat runtime-close-1.state

{"accounts":[[[{"address":"addr1vywt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0czta9gx"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":0}


## Run the Contract

### Transaction 1. Create the ontract

Build the transaction.

In [15]:
TX_1=$(
marlowe create \
  --core-file runtime-close-1.contract \
  --min-utxo "$MINIMUM_ADA" \
  --change-address "$PARTY_ADDR" \
  --manual-sign runtime-close-1.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
CONTRACT_ID="$TX_1"
echo "CONTRACT_ID = TX_1 = $CONTRACT_ID"

CONTRACT_ID = TX_1 = 44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b#1


Sign the transaction.

In [16]:
cardano-cli transaction sign \
  --tx-body-file runtime-close-1.txbody \
  --out-file     runtime-close-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [17]:
marlowe submit runtime-close-1.tx | jq 'fromjson' | json2yaml

blockHeaderHash: 59593fb1f17aa71347dd49b65081bc99581ff1fbcbc732258f6fb9029d25d9f5
blockNo: 8126304
slotNo: 79201949


View the contract's UTxO.

In [18]:
CONTRACT_ADDR=$(marlowe-cli contract address "${MAGIC[@]}")
echo "$CONTRACT_ADDR"

addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf


In [19]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_1//#*/}/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "44a946ba1ee21f1a57a99527d8698a828217268264bacc10f9f705bcb726ad48"


Watch the contract.

In [20]:
marlowe add "$CONTRACT_ID"

44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b#1


View the contract's history.

In [21]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b (creation)
ContractId:      44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b#1
SlotNo:          79201949
BlockNo:         8126304
BlockId:         59593fb1f17aa71347dd49b65081bc99581ff1fbcbc732258f6fb9029d25d9f5
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    Close



### Transaction 2. Party deposits loan amount

The party deposits the loan amount.

In [22]:
RIGHT_NOW="$(($(date -u +%s) * SECOND))"
TX_2=$(
marlowe advance \
  --contract "$CONTRACT_ID" \
  --validity-lower-bound "$((RIGHT_NOW-1*MINUTE))" \
  --validity-upper-bound "$((RIGHT_NOW+5*MINUTE))" \
  --change-address "$PARTY_ADDR" \
  --manual-sign runtime-close-2.txbody \
| sed -e 's/^.*"\([^\\]*\)\\.*$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = bb4823905e187678a96f09df06b5b3489bab3f779a4bebb7b807e5fbf3bc2f60


Sign the transaction.

In [23]:
cardano-cli transaction sign \
  --tx-body-file runtime-close-2.txbody \
  --out-file     runtime-close-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [24]:
marlowe submit runtime-close-2.tx | jq 'fromjson' | json2yaml

blockHeaderHash: ad80feb87b0f9bb5f487c0f4b4643bf90909720ac54474f7283230b3c2a619b8
blockNo: 8126319
slotNo: 79202332


We see that the contract did indeed close.

In [25]:
marlowe log --show-contract "$CONTRACT_ID"

transaction 44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b (creation)
ContractId:      44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b#1
SlotNo:          79201949
BlockNo:         8126304
BlockId:         59593fb1f17aa71347dd49b65081bc99581ff1fbcbc732258f6fb9029d25d9f5
ScriptAddress:   addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf
Marlowe Version: 1

    Close

transaction bb4823905e187678a96f09df06b5b3489bab3f779a4bebb7b807e5fbf3bc2f60 (close)
ContractId: 44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b#1
SlotNo:     79202332
BlockNo:    8126319
BlockId:    ad80feb87b0f9bb5f487c0f4b4643bf90909720ac54474f7283230b3c2a619b8
Inputs:     []




The contract address is empty.

In [26]:
cardano-cli query utxo "${MAGIC[@]}" --address "$CONTRACT_ADDR" | sed -n -e "1,2p;/${TX_1//#*/}/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


The party received their 2 Ada.

In [27]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bb4823905e187678a96f09df06b5b3489bab3f779a4bebb7b807e5fbf3bc2f60     0        7396516 lovelace + TxOutDatumNone
bb4823905e187678a96f09df06b5b3489bab3f779a4bebb7b807e5fbf3bc2f60     1        2000000 lovelace + TxOutDatumNone


## Cleanup

Remove the contract from history tracking.

In [28]:
marlowe rm "$CONTRACT_ID"

44649bd1cb8ba45f9a457703ec3d4b183f898034e22cea196bc019bca7e4769b#1


In [29]:
marlowe ls

Pay the funds back to the faucet.

In [30]:
marlowe-cli transaction simple \
  "${MAGIC[@]}" \
  --tx-in bb4823905e187678a96f09df06b5b3489bab3f779a4bebb7b807e5fbf3bc2f60#0 \
  --tx-in bb4823905e187678a96f09df06b5b3489bab3f779a4bebb7b807e5fbf3bc2f60#1 \
  --change-address $FAUCET_ADDR \
  --required-signer $PARTY_SKEY \
  --out-file /dev/null \
  --submit 600

TxId "12219102ef798d9d2a2c09664aceda5c7eb496af5cf67b7d9642bcdb3e07d813"


See that the funds have been returned to the faucet.

In [31]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
